# Zero-shot NLI based ACE Relation Extraction

In this lab we will implement a model that performs zero-shot Relation Extraction on ACE dataset ([Sainz et al. 2020](https://aclanthology.org/2021.emnlp-main.92/)). The approach relies on label verbalization using hand-crafted templates and pre-trained entailment model to infer the relation of given entities ($e_1$ and $e_2$) in a particular context $c$.

![](https://drive.google.com/uc?id=1nQUMDZRcWOczI1Jp9rTpIOvy3uutjzji)


Two main components are required to implement the model.

__Verbalizing relations as hypothesis__: A function that generate hypothesis for relation given manually defined templates and a pair of entities ($e_1$, $e_2$).

![](https://drive.google.com/uc?id=1q3I0f5_5un2oRLAA_lnkb4p4JQzCF8Gf)

__NLI for inferring relations__: In a second step we make use of the NLI model to infer the relation
label. Given context $c$ as premise and verbalized hypothesis, the systems returns the entailment probability between them. Relation is inferred from the set of possible relation labels with the highest entailment probability.

![](https://drive.google.com/uc?id=1jH6gr-SFtvAjEqH-IT8fv7HNuEHYMCOa)



__Ask2Transformers__ In this notebook we will use the framework (`a2t` for short) develop by the main author of the paper ([Sainz et al. 2020](https://aclanthology.org/2021.emnlp-main.92/)) and available in github: 

- https://github.com/osainz59/Ask2Transformers
  

__ACE dataset__  We will be using an extendedn version of ACE in this notebook. Note that if you want to run and publish experiments you need to have a LDC license for it (https://www.ldc.upenn.edu/collaborations/past-projects/ace). You can check the ACE Annotation guidelines for relation extraction in the following link:

- https://www.ldc.upenn.edu/sites/www.ldc.upenn.edu/files/english-relations-guidelines-v6.2.pdf



## Setting up

In [ ]:
# Mount Drive files
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install git+https://github.com/osainz59/Ask2Transformers.git
!pip install sentencepiece 

## Load ACE dataset and inspect it
In this section we are goint to load ACE and create `ACERelationClassificationDataset` object that is the __Data__ component requiered in the `a2t` framework. The code that create the data object is given to you, so only need to inspect the loaded ACE data. 

In [ ]:
from typing import List
import json

from a2t.data import Dataset
from a2t.tasks import BinaryTask, BinaryFeatures
from a2t.tasks import RelationClassificationTask, RelationClassificationFeatures

class ACERelationClassificationDataset(Dataset):
    """A class to handle ACE dataset.

    This class converts ACE data files into a list of `BinaryFeatures`.
    """

    def __init__(self, input_path: str, labels: List[str], *args, **kwargs) -> None:
        super().__init__(labels=labels, *args, **kwargs)

        instances = self.loadACE(input_path)
        for example in instances:
            entities = example['graph']['entities']
            if len(example['graph']['relations']) > 0:
                context = ' '.join(example['tokens'])
                for re in example['graph']['relations']:
                    start = entities[re[0]][0]
                    end = entities[re[0]][1]
                    X_type = entities[re[0]][2]
                    X = example['tokens'][start:end]
                    start = entities[re[1]][0]

                    end = entities[re[1]][1]
                    Y_type = entities[re[1]][2]
                    Y = example['tokens'][start:end]
                    rel_type = re[2]
                    self.append(BinaryFeatures(X=X, Y=Y, context=context, inst_type=f"{X_type}:{Y_type}",label=rel_type))

    def load_json(self, file_path):
        with open(file_path, 'rt') as f:
            data = [json.loads(line) for line in f]
        return data

    def loadACE(self, file_path):
        instances = self.load_json(file_path)
        new_instances = []
        for inst in instances:
            doc_id = inst['doc_id']
            sent_id = inst['sent_id']
            tokens = inst['tokens']
            entity2id = {ent['id']: i for i, ent in enumerate(inst['entity_mentions'])}
            entities = [
                [ent['start'], ent['end'], ent['entity_type'], ent['mention_type'], 1.0]
                for ent in inst['entity_mentions']
            ]
            triggers, roles = [], []
            for i, event in enumerate(inst['event_mentions']):
                triggers.append(
                    [event['trigger']['start'], event['trigger']['end'], event['event_type'], 1.0]
                )
                for arg in event['arguments']:
                    roles.append(
                        [ i, entity2id[arg['entity_id']], arg['role'], 1.0 ]    
                    )
            relations = []
            for i, relation in enumerate(inst['relation_mentions']):
                relations.append([entity2id[relation['arguments'][0]['entity_id']],
                                    entity2id[relation['arguments'][1]['entity_id']],
                                    relation['relation_subtype']])

            new_instances.append({
                'doc_id': doc_id,
                'sent_id': sent_id,
                'tokens': tokens,
                'graph': {
                    'entities': entities,
                    'triggers': triggers,
                    'relations': relations,
                    'roles': roles
                }
            })
        return new_instances

Next we need to specify the set of the relation existing in the dataset. ACE only annotates positive relation and we include the `NO-Relation` label in order to infer the cases that given 2 entities and context no positive relation is held. As verbalization of the relation types is incomplete in the development of the verabilazation templates, the use of `NO-Relation` allows a more effective way of measuring template accuracy.

In [ ]:
labels = ["NO-Relation",
          "ART:User-Owner-Inventor-Manufacturer", 
          "GEN-AFF:Citizen-Resident-Religion-Ethnicity",
          "GEN-AFF:Org-Location",
          "ORG-AFF:Employment",
          "ORG-AFF:Founder",
          "ORG-AFF:Investor-Shareholder", 
          "ORG-AFF:Membership",
          "ORG-AFF:Ownership",
          "ORG-AFF:Sports-Affiliation",
          "ORG-AFF:Student-Alum",
          "PART-WHOLE:Artifact",
          "PART-WHOLE:Geographical",
          "PART-WHOLE:Subsidiary",
          "PER-SOC:Business",
          "PER-SOC:Family",
          "PER-SOC:Lasting-Personal",
          "PHYS:Located",
          "PHYS:Near"
        ]

Speficy the valid conditions of each relation type. This will be usefull to posprocess some incorrect predictions. 

In [ ]:
# define valid conditions
valid_conditions = {
    "ART:User-Owner-Inventor-Manufacturer" : [
        "PER:FAC", 
        "ORG:FAC", 
        "GPE:FAC"
    ], 
    "GEN-AFF:Citizen-Resident-Religion-Ethnicity": [
        "PER:PER",
        "PER:LOC",
        "PER:GPE",
        "PER:ORG"
    ],
    "GEN-AFF:Org-Location": [
        "ORG:LOC",
        "ORG:GPE"                             
    ],
    "ORG-AFF:Employment": [
        "PER:PER",
        "PER:ORG"
    ],
    "ORG-AFF:Founder": [
        "PER:ORG",
        "PER:GPE",
        "ORG:ORG",
        "ORG:GPE"
    ],
    "ORG-AFF:Investor-Shareholder": [
        "PER:ORG",
        "PER:GPE",
        "ORG:ORG",
        "ORG:GPE",
        "GPE:ORG",
        "GPE:GPE"
    ], 
    "ORG-AFF:Membership": [
        "PER:ORG",
        "ORG:ORG",
        "GPE:ORG"
    ],
    "ORG-AFF:Ownership": [
        "PER:ORG"
    ],
    "ORG-AFF:Sports-Affiliation": [
        "PER:ORG"
    ],
    "ORG-AFF:Student-Alum": [
        "PER:ORG"
    ],
    "PART-WHOLE:Artifact": [
        "WEA:WEA",
        "WEA:VEH",
        "VEH:VEH"
    ],
    "PART-WHOLE:Geographical": [
        "FAC:FAC", 
        "FAC:LOC",
        "FAC:GPE",
        "LOC:FAC",
        "LOC:LOC",
        "LOC:GPE",
        "GPE:FAC",
        "GPE:LOC",
        "GPE:GPE"
    ],
    "PART-WHOLE:Subsidiary": [
        "ORG:ORG",
        "ORG:GPE"
    ],
    "PER-SOC:Business": [
        "PER:PER"
    ],
    "PER-SOC:Family": [
        "PER:PER"
    ],
    "PER-SOC:Lasting-Personal": [
        "PER:PER"
    ],
    "PHYS:Located" : [
        "PER:FAC",
        "PER:LOC",
        "PER:GPE"
    ],
    "PHYS:Near" : [
        "PER:FAC",
        "PER:GPE",
        "PER:LOC",
        "FAC:FAC",
        "FAC:GPE",
        "FAC:LOC",
        "GPE:FAC",
        "GPE:GPE",
        "GPE:LOC",
        "LOC:FAC", 
        "LOC:GPE",
        "LOC:LOC"
    ]
}

Load the data.

In [ ]:
file_path = "drive/MyDrive/Colab Notebooks/nlp-app-II/data/ace-e+/train.oneie.json"
dataset = ACERelationClassificationDataset(file_path, labels)

### Exercise 1
- Inspect and understand the relations annoated in ACE. For tha you can use the function `inspect_relatio` that given a relation type and the dataset it prints some examples. 

- Understanding the meaning of the relation type is key to create good verbalization templetes. 

In [ ]:
import textwrap

# create a function to show examples of given relation type
def inspect_relation(relation_type, dataset, top_k=10):
    result = [datum for datum in dataset if relation_type == datum.label]
    for i in range(top_k):
        datum = result[i]
        print (" ".join(datum.X) + " - " + datum.label + " - " + " ".join(datum.Y) + " ("+ datum.inst_type + "):")
        print(textwrap.fill(datum.context, 80))
        print()

In [ ]:
inspect_relation("PART-WHOLE:Artifact", dataset)

## Load entailment model

In [ ]:
from a2t.base import EntailmentClassifier

nlp = EntailmentClassifier(
    'roberta-large-mnli',
    use_cuda=True
)

## Design of templates

Once we have set the labels, defined the valid conditions for each relation type, and load the entailment model, we can start defining verbalization templates that generate the hypothesis that will be evaluated in the entailment model. 

The idea of designing the template is to create in quick way some set of templates and evaluate them in an iterative way. This part of the process should be as simple as possible and as fun as possible. The iterative process is as follows:

1. Select a relation type.
2. Inspect and understand the relation type and think of ways of verbalizing it (kind of prototypical example).
3. Write a least one template. Don't think too much. The idea is try different simple verbalization.
4. Evaluate the model on a dataset (typically a small one) using the current set of templates. 
5. Chech if new templates is working correctly. 
6. If new templates are working, select a new relation type and repeate the process.
7. If it is not working think of new templates, write them and evaluate them. 

## Extract smaller set of the dataset

Current dataset is quite large to run the evaluation manually. When generating templates for relation extraction, it is enough to have a few example per relation (those that we can find in the annotation guidelines). In zero-shot scenario we do not have any annotated example. We will simulate this scenario by taking only 2 examples for each relation type. 

In [ ]:
import numpy as np

def get_smaller_dataset(dataset, labels):
    small = []
    for label in labels:
        if label != "NO-Relation":
            i = 0
            j = 0
            while (i < 2 and j < len(dataset)):
                if label == dataset[j].label:
                    small.append(dataset[j])
                    i += 1
                j += 1
    labels = np.array([dataset.labels2id[datum.label] for datum in small])
    return small, labels

small_data, small_labels = get_smaller_dataset(dataset, labels)

### Exercise 2
- the code below shows some examples that verbalize `ART:User-Owner-Inventor-Manufacturer`  and `PART-WHOLE:Geographical` relation types. As you can see templates are quite simple. For example, `ART:User-Owner-Inventor-Manufacturer` can be describes with templates like "_X os the owner of Y_".
- Follow the instrucction explained above to generate the templates for the rest of the remaining types.

In [ ]:
# define templates
templates = {
    "ART:User-Owner-Inventor-Manufacturer": [
        "{X} is the owner of {Y}",
        "{X} have {Y}",
        "{X} invented {Y}",                                              
    ],
#    "GEN-AFF:Citizen-Resident-Religion-Ethnicity": [],
#    "GEN-AFF:Org-Location": [],
#    "ORG-AFF:Employment": [],
#    "ORG-AFF:Founder": [],
#    "ORG-AFF:Investor-Shareholder" : [], 
#    "ORG-AFF:Membership": [],
#    "ORG-AFF:Ownership": [],
#    "ORG-AFF:Sports-Affiliation": [],
#    "ORG-AFF:Student-Alum": [],
#    "PART-WHOLE:Artifact"
    "PART-WHOLE:Geographical": [
        "{X} is in {Y}",
        "{X} is located in {Y}"
    ],
#    "PART-WHOLE:Subsidiary": [],
#    "PER-SOC:Business": [],
#    "PER-SOC:Family": [],
#    "PER-SOC:Lasting-Personal": [],
#    "PHYS:Located": [],
#    "PHYS:Near": [],
}

The following task defines the task we want to solve specifying the templates, and valid conditions, among other stuff. 

You need to re-run the code for every time you update the templates. 

In [ ]:
task = RelationClassificationTask(
    name="ACE Relation Classification task",
    required_variables=["X", "Y"],
    additional_variables=["inst_type"],
    labels=labels,
    templates=templates,
    valid_conditions=valid_conditions,
    negative_label_id=0,
    multi_label=True,
    features_class=BinaryFeatures
)

## Run inference

In [ ]:
preds, proba_preds = nlp(task=task, features=small_data, return_labels=True, return_confidences=True, return_raw_output=True)

Show the inferred predictions

In [ ]:
preds

In [ ]:
small_labels

Evalute the templates on the smaller dataset

In [ ]:
task.compute_metrics(small_labels, proba_preds, "optimize")

# Evaluate your model in the test set


In [ ]:
test_path = "/content/drive/MyDrive/00-Irakaskuntza/HAP-LAP-masterra/NLP-Applications-2/Part1: Information-extraction/notebooks/data/ace-e+/test.oneie.json"
testset = ACERelationClassificationDataset(test_path, labels)
test_preds, test_proba_preds = nlp(task=task, features=testset, return_labels=True, return_confidences=True, return_raw_output=True)

In [ ]:
task.compute_metrics(testset.labels, test_proba_preds, "optimize")